In [1]:
#Second test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [2]:
# Global Setup
# Creating temp local repos to test on
TMPA=/tmp/temp-git-test

Cloning into '/tmp/temp-git-test'...
remote: Enumerating objects: 529, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 529 (delta 85), reused 88 (delta 49), pack-reused 405
Receiving objects: 100% (529/529), 46.09 KiB | 963.00 KiB/s, done.
Resolving deltas: 100% (322/322), done.
From https://github.com/LorenzoFerraces/bash-test
 * branch            main       -> FETCH_HEAD
Already up to date.
[main 968fb56] first commit on second local
 1 file changed, 1 insertion(+)
 create mode 100644 hello.txt
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 4 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 312 bytes | 312.00 KiB/s, done.
Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   abc709b..968fb56  main -> main


In [ ]:
# Global setup
# Define aliases for encoding output (e.g., to ensure output the same across runs).
## OLD: alias encode-control-chars='perl -pe "s/\t/<tab>/g;  s/\n/<nl>/g;  s/[\x00-\x31]/<cc>/g;"'
alias encode-control-chars='perl -pe "s/\t/<tab>/g;  s/[\x00-\x08\x11\x31]/<cc>/g;"'
alias encode-nums='perl -pe "s/[0-9]+/N/g;"'
## TODO: define a better hex encoder to avoid false positives (e.g., abc).
alias encode-hex='perl -pe "s/[0-9A-F]{4,}/<hex>/ig;"'
# note: uses perl slurp mode (i.e., read entire file) to check for consecutive newlines
alias encode-newlines='perl -0777 -pe "while (/\n\n/) { s/\n\n/\n<newline>\n/g; }"'
function encode-text () { encode-control-chars | encode-nums | encode-hex | encode-newlines; }

In [ ]:
git clone https://github.com/LorenzoFerraces/bash-test.git "$TMPA" > /dev/null 2>&1

In [ ]:
# Move to second local to push some changes
command cd "$TMPA" > /dev/null 2>&1
git pull origin main > /dev/null 2>&1
echo "hello second local" > hello.txt
git add hello.txt > /dev/null 2>&1
git commit -m "first commit on second local" > /dev/null 2>&1
git push > /dev/null 2>&1

In [3]:
# git-rename-file tests
# Assert file doesn't exist
[ ! -f "newHello.txt" ]; echo $?;

0


In [4]:
export GIT_MESSAGE="renamed hello.txt"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
contents="$(cat hello.txt)"
git-rename-file hello.txt newHello.txt 2>&1 | encode-text

issuing: git mv hello.txt newHello.txt
issuing: git add newHello.txt

GIT_NO_CONFIRM set to 1, so skipping confirmation
issuing: git commit -m 'renamed hello.txt'
    [main 7606053] renamed hello.txt
     1 file changed, 0 insertions(+), 0 deletions(-)
     rename hello.txt => newHello.txt (100%)
GIT_NO_CONFIRM set to 1, so skipping confirmation
issuing: git push --verbose
Pushing to https://github.com/LorenzoFerraces/bash-test.git
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 4 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (2/2), 279 bytes | 279.00 KiB/s, done.
Total 2 (delta 0), reused 0 (delta 0), pack-reused 0
POST git-receive-pack (452 bytes)
To https://github.com/LorenzoFerraces/bash-test.git
   968fb56..7606053  main -> main
updating local tracking ref 'refs/remotes/origin/main'
[main 7606053] renamed hello.txt
 1 file changed, 0 insertions(+), 0 deletions(-)
 rename hello.txt => newHello.txt (100%)

issuin

In [5]:
# Asserts
[ -f "newHello.txt" ] && [ "$(cat newHello.txt)" == "$(echo $contents)" ]; 
echo $?;

0


In [6]:
# git-restore-staged-alias test
echo "second change on local" >> newHello.txt
git add newHello.txt > /dev/null 2>&1

In [7]:
# Assert theres is one staged file 
[ $(git status -s | grep -c "^[MTADRCU]") -eq 1 ]; echo $?; 

0


In [8]:
export GIT_MESSAGE="restored newHello.txt"
export GIT_NO_CONFIRM="1"
git-restore-staged-alias newHello.txt 2>&1 | encode-text
rm -rf _git-trash > /dev/null

issuing: cp -vpf newHello.txt _git-trash
issuing: git restore --staged newHello.txt
'newHello.txt' -> '_git-trash/newHello.txt'
removed '_git-trash/newHello.txt'
removed directory '_git-trash'


In [9]:
# Assert theres are no staged files
[ $(git status -s | grep -c "^[MTADRCU]") -eq 0 ]; echo $?; 

0


In [10]:
# git-revert-file-alias tests
echo "second change on local" >> newHello.txt


In [11]:
# Assert theres is 1 unstaged file
[ $(git status -s | grep -c "^\s[MTADRCU]") -eq 1 ]; echo $?; 

0


In [12]:
#revert unstaged file
export GIT_MESSAGE="reverted newHello.txt"
export GIT_NO_CONFIRM="1"
git-revert-file-alias newHello.txt 2>&1 | encode-text
rm -rf _git-trash > /dev/null 2>&1

issuing: cp -vpf newHello.txt _git-trash
issuing: git reset HEAD newHello.txt
issuing: git checkout -- newHello.txt
issuing: git stash list
'newHello.txt' -> '_git-trash/newHello.txt'
Unstaged changes after reset:
M	newHello.txt
removed '_git-trash/newHello.txt'
removed directory '_git-trash'


In [13]:
# Assert file was reverted
# this matches any files except untracked
# read https://git-scm.com/docs/git-status#_short_formatz
[ $(git status -s | grep -c "^\s*[MTADRCU]") -eq 0 ]; echo $?; 

0


In [14]:
# git-move-to-dir tests
mkdir folder
git add folder
git-move-to-dir "$TMP/folder" newHello.txt | true
command cd folder

/tmp/temp-git-test/folder


In [15]:
#Assert the file has been moved to folder
[ $(ls | wc -l) -eq 1 ]; echo $?;

1


In [16]:
# git-conflicts-alias tests
command cd $TMPA

0


In [ ]:
# first assert there are no conflicts
[ $(git-conflicts-alias | wc -l) -eq 0 ]; echo $?

In [17]:
# create branches that conflict later
git branch branch-a
git branch branch-b


Switched to branch 'branch-a'
[branch-a 545f76f] first commit branch-a
 1 file changed, 1 insertion(+)
 create mode 100644 conflicts.txt
Switched to branch 'branch-b'
[branch-b 898caf3] first commit branch-b
 1 file changed, 1 insertion(+)
 create mode 100644 conflicts.txt


In [ ]:
git checkout branch-a
echo "line a" >> conflicts.txt
git add . > /dev/null 2>&1
git commit -m "first commit branch-a" > /dev/null 2>&1

In [ ]:
git switch branch-b
echo "line b" >> conflicts.txt
git add . > /dev/null 2>&1
git commit -m "first commit branch-b" > /dev/null 2>&1

In [18]:
# merge branches 
git switch main
git merge branch-a > /dev/null 2>&1
git merge branch-b | true > /dev/null 2>&1

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating 7606053..545f76f
Fast-forward
 conflicts.txt | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 conflicts.txt


In [19]:
# assert there are now conflicts 
[ $(git-conflicts-alias | wc -l) -gt 0 ]; echo $?;

0


In [20]:
# undo changes and remove newly created files

# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'
rm -f *.txt > /dev/null 2>&1
git rm -f *.txt > /dev/null 2>&1
git rm -rf folder 2>&1
git add . > /dev/null 2>&1


fatal: pathspec 'folder' did not match any files
[main 0f30a78] cleanup test files
 2 files changed, 2 deletions(-)
 delete mode 100644 conflicts.txt
 delete mode 100644 newHello.txt
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (5/5), 542 bytes | 542.00 KiB/s, done.
Total 5 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   7606053..0f30a78  main -> main


In [ ]:
## Lorenzo-TODO2: use one cell per output (to make this less brittle)
git commit -m "cleanup test files" > /dev/null 2>&1
git push > /dev/null 2>&1

In [21]:
## OLD: extraneous command as full path can be used as below
## # delete test local repos
## command cd "/tmp"

In [ ]:
## OLD:
## rm -rf temp-git-test > /dev/null
## Lorenzo-Note: useful for retaining directories for post-hoc analysis
rename-with-file-date /tmp/temp-git-test > /dev/null
export GIT_NO_CONFIRM=$previous